# AC2 - Análise de Vibração de Rolamentos
## Processamento Digital de Sinais

Este notebook implementa técnicas de Processamento Digital de Sinais (PDS) aplicadas a dados reais de vibração de motores, simulando o fluxo típico de um sistema de monitoramento industrial.

### Objetivos:
1. Coleta e obtenção de dados
2. Tratamento do sinal (remoção de offset, normalização, filtragem)
3. Análise espectral (FFT)
4. Extração de features (RMS, Peak, Crest Factor)
5. Conexão com aplicações de Indústria 4.0


In [ ]:
# Importação de bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq
from scipy.signal import butter, filtfilt, find_peaks
import os
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
# Tenta usar estilo seaborn, caso contrário usa padrão
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('default')
        plt.rcParams['axes.grid'] = True
        plt.rcParams['grid.alpha'] = 0.3

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

# Configuração para salvar gráficos
OUTPUT_DIR = 'output/graficos'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✓ Pasta de saída criada: {OUTPUT_DIR}")

# Configuração de qualidade dos gráficos
DPI = 300  # Resolução para salvamento
plt.rcParams['savefig.dpi'] = DPI
plt.rcParams['savefig.bbox'] = 'tight'  # Remove espaços em branco
plt.rcParams['savefig.format'] = 'png'  # Formato padrão

print("Bibliotecas importadas com sucesso!")
print(f"✓ Gráficos serão salvos em: {OUTPUT_DIR}/")
print(f"NumPy versão: {np.__version__}")
print(f"Pandas versão: {pd.__version__}")


## 1. Coleta e Obtenção de Dados

Nesta etapa, vamos carregar o dataset de vibração de rolamentos. O dataset pode estar em formato CSV, TXT ou similar.


In [ ]:
# Função para carregar dataset de vibração em diferentes formatos
def load_bearing_dataset(data_path='data', sample_rate=12000):
    """
    Carrega dataset de vibração de rolamentos, tentando diferentes formatos comuns.
    
    Formatos suportados:
    - CSV com cabeçalho (time, vibration ou colunas X, Y, Z)
    - CSV sem cabeçalho (apenas valores numéricos)
    - TXT com valores separados por espaço/vírgula
    - Arquivos MAT (MATLAB)
    
    Retorna: DataFrame com colunas 'time' e 'vibration'
    """
    import glob
    
    # Lista de padrões de arquivos para procurar
    patterns = [
        os.path.join(data_path, '**', '*.csv'),
        os.path.join(data_path, '**', '*.txt'),
        os.path.join(data_path, '**', '*.dat'),
        os.path.join(data_path, '*.csv'),
        os.path.join(data_path, '*.txt'),
        os.path.join(data_path, '*.dat'),
    ]
    
    files_found = []
    for pattern in patterns:
        files_found.extend(glob.glob(pattern, recursive=True))
        if files_found:
            break
    
    if not files_found:
        print("⚠ Nenhum arquivo de dados encontrado. Usando dados sintéticos.")
        return None
    
    # Tentar carregar o primeiro arquivo encontrado
    file_path = files_found[0]
    print(f"📁 Tentando carregar: {file_path}")
    
    # Tentar diferentes métodos de carregamento
    methods = [
        # Método 1: CSV com cabeçalho
        lambda: pd.read_csv(file_path),
        # Método 2: CSV sem cabeçalho, com delim whitespace
        lambda: pd.read_csv(file_path, header=None, delim_whitespace=True),
        # Método 3: CSV com vírgula
        lambda: pd.read_csv(file_path, header=None, sep=','),
        # Método 4: TXT com espaço (usar delim_whitespace)
        lambda: pd.read_csv(file_path, header=None, delim_whitespace=True),
    ]
    
    for i, method in enumerate(methods, 1):
        try:
            df = method()
            print(f"✓ Arquivo carregado usando método {i}")
            
            # Processar DataFrame
            if df.shape[1] == 1:
                # Apenas uma coluna - assumir que é o sinal
                df.columns = ['vibration']
                # Criar tempo baseado na taxa de amostragem
                df['time'] = np.arange(len(df)) / sample_rate
            elif df.shape[1] == 2:
                # Duas colunas - assumir [tempo, vibração]
                if df.columns[0] == 'time' or df.columns[0] == 0:
                    df.columns = ['time', 'vibration']
                else:
                    df.columns = ['time', 'vibration']
            elif df.shape[1] >= 3:
                # Múltiplas colunas - pode ser X, Y, Z ou múltiplos canais
                print(f"  Encontradas {df.shape[1]} colunas. Usando primeira coluna como sinal.")
                if 'time' in str(df.columns[0]).lower() or 't' in str(df.columns[0]).lower():
                    df = df.iloc[:, [0, 1]]
                    df.columns = ['time', 'vibration']
                else:
                    # Usar primeira coluna como vibração, criar tempo
                    df = pd.DataFrame({
                        'vibration': df.iloc[:, 0].values,
                        'time': np.arange(len(df)) / sample_rate
                    })
            
            # Garantir que temos time e vibration
            if 'time' not in df.columns:
                df['time'] = np.arange(len(df)) / sample_rate
            if 'vibration' not in df.columns:
                df['vibration'] = df.iloc[:, 0]
            
            # Converter para float
            df['time'] = pd.to_numeric(df['time'], errors='coerce')
            df['vibration'] = pd.to_numeric(df['vibration'], errors='coerce')
            
            # Remover NaN
            df = df.dropna()
            
            print(f"✓ Dados processados: {len(df)} amostras")
            return df
            
        except Exception as e:
            continue
    
    print("⚠ Não foi possível carregar o arquivo. Usando dados sintéticos.")
    return None


# Configuração de parâmetros
DATA_PATH = 'data'  # Caminho para os dados
SAMPLE_RATE = 12000  # Taxa de amostragem em Hz (ajuste conforme seu dataset)
NYQUIST_FREQ = SAMPLE_RATE / 2  # Frequência de Nyquist

# Tentar carregar o dataset real
df = load_bearing_dataset(DATA_PATH, SAMPLE_RATE)

# Se não encontrou dados reais, usar dados sintéticos para demonstração
if df is None:
    print("\n" + "="*60)
    print("Gerando dados sintéticos para demonstração...")
    print("NOTA: Para usar dados reais, extraia o dataset e coloque em data/")
    print("="*60 + "\n")
    
    # Simulação de sinal de vibração com múltiplas componentes
    duration = 1.0  # segundos
    t = np.linspace(0, duration, int(SAMPLE_RATE * duration), endpoint=False)
    
    # Componentes de frequência típicas de falhas em rolamentos
    freq_rotation = 30  # Hz - frequência de rotação
    freq_bpfo = 3.58 * freq_rotation  # Ball Pass Frequency Outer race
    freq_bpfi = 5.41 * freq_rotation  # Ball Pass Frequency Inner race
    
    # Sinal sintético com ruído
    signal_raw = (
        0.5 * np.sin(2 * np.pi * freq_rotation * t) +
        0.3 * np.sin(2 * np.pi * freq_bpfo * t) +
        0.2 * np.sin(2 * np.pi * freq_bpfi * t) +
        0.1 * np.sin(2 * np.pi * 60 * t) +  # Ruído de linha (60 Hz)
        0.05 * np.random.randn(len(t)) +  # Ruído branco
        0.2  # Offset DC
    )
    
    # Criar DataFrame para compatibilidade
    df = pd.DataFrame({
        'time': t,
        'vibration': signal_raw
    })
    
    print(f"✓ Dados sintéticos gerados: {len(df)} amostras")
else:
    print(f"✓ Dados reais carregados com sucesso!")

print(f"✓ Duração: {df['time'].max():.2f} segundos")
print(f"✓ Taxa de amostragem estimada: {SAMPLE_RATE} Hz")
print(f"✓ Número de amostras: {len(df)}")


In [ ]:
# Verificação da estrutura dos dados
print("Estrutura do dataset:")
print(df.head())
print(f"\nDimensões: {df.shape}")
print(f"\nInformações do dataset:")
print(df.info())
print(f"\nEstatísticas descritivas:")
print(df.describe())


In [ ]:
# Extrair sinal e tempo
t = df['time'].values
signal_raw = df['vibration'].values

print(f"Sinal extraído: {len(signal_raw)} amostras")
print(f"Amplitude mínima: {np.min(signal_raw):.4f}")
print(f"Amplitude máxima: {np.max(signal_raw):.4f}")
print(f"Amplitude média: {np.mean(signal_raw):.4f}")


## 2. Tratamento do Sinal

### 2.1 Visualização do Sinal Bruto


In [ ]:
# Visualização do sinal bruto no tempo
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(t[:int(SAMPLE_RATE*0.1)], signal_raw[:int(SAMPLE_RATE*0.1)], 'b-', linewidth=0.8, alpha=0.7)
ax.set_xlabel('Tempo (s)', fontsize=12)
ax.set_ylabel('Amplitude', fontsize=12)
ax.set_title('Sinal Bruto de Vibração (Primeiros 100ms)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='k', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '01_sinal_bruto.png'), dpi=DPI, bbox_inches='tight')
print(f"✓ Gráfico salvo: {OUTPUT_DIR}/01_sinal_bruto.png")
plt.show()

print(f"Estatísticas do sinal bruto:")
print(f"  Média (offset DC): {np.mean(signal_raw):.4f}")
print(f"  Desvio padrão: {np.std(signal_raw):.4f}")
print(f"  Valor RMS: {np.sqrt(np.mean(signal_raw**2)):.4f}")


### 2.2 Remoção de Offset (Componente DC)


In [ ]:
# Remoção do offset DC (subtração da média)
dc_offset = np.mean(signal_raw)
signal_no_offset = signal_raw - dc_offset

print(f"Offset DC removido: {dc_offset:.4f}")
print(f"Média após remoção: {np.mean(signal_no_offset):.6f}")


### 2.3 Normalização


In [ ]:
# Normalização pela amplitude máxima (normalização para [-1, 1])
max_amplitude = np.max(np.abs(signal_no_offset))
signal_normalized = signal_no_offset / max_amplitude if max_amplitude > 0 else signal_no_offset

print(f"Amplitude máxima antes da normalização: {max_amplitude:.4f}")
print(f"Amplitude máxima após normalização: {np.max(np.abs(signal_normalized)):.4f}")


### 2.4 Filtragem do Sinal

Aplicaremos três tipos de filtros:
- **Filtro Passa-Baixas**: Remove frequências acima de um limite (anti-aliasing)
- **Filtro Passa-Altas**: Remove frequências muito baixas (drift)
- **Filtro Notch**: Remove frequência específica (ex: 60 Hz da rede elétrica)


In [ ]:
# Filtro Passa-Baixas (Low-pass filter)
# Remove frequências acima de 5000 Hz (ajuste conforme necessário)
cutoff_low = 5000  # Hz
b_low, a_low = butter(4, cutoff_low / NYQUIST_FREQ, btype='low')
signal_lowpass = filtfilt(b_low, a_low, signal_normalized)

print(f"✓ Filtro passa-baixas aplicado (cutoff: {cutoff_low} Hz)")


In [ ]:
# Filtro Passa-Altas (High-pass filter)
# Remove frequências abaixo de 10 Hz (remoção de drift)
cutoff_high = 10  # Hz
b_high, a_high = butter(4, cutoff_high / NYQUIST_FREQ, btype='high')
signal_highpass = filtfilt(b_high, a_high, signal_lowpass)

print(f"✓ Filtro passa-altas aplicado (cutoff: {cutoff_high} Hz)")


In [ ]:
# Filtro Notch (para remover 60 Hz da rede elétrica)
notch_freq = 60  # Hz
quality_factor = 30  # Fator de qualidade (largura do notch)
b_notch, a_notch = signal.iirnotch(notch_freq, quality_factor, SAMPLE_RATE)
signal_notch = filtfilt(b_notch, a_notch, signal_highpass)

print(f"✓ Filtro notch aplicado (frequência removida: {notch_freq} Hz)")

# Sinal final tratado
signal_treated = signal_notch.copy()

print(f"\n✓ Tratamento completo finalizado!")
print(f"  Sinal original: média={np.mean(signal_raw):.4f}, std={np.std(signal_raw):.4f}")
print(f"  Sinal tratado: média={np.mean(signal_treated):.6f}, std={np.std(signal_treated):.4f}")


### 2.5 Comparação: Sinal Bruto vs Tratado


In [ ]:
# Gráfico comparativo: Sinal bruto vs tratado
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Primeiros 200ms para visualização
time_window = int(SAMPLE_RATE * 0.2)
t_window = t[:time_window]

# Sinal bruto
axes[0].plot(t_window, signal_raw[:time_window], 'b-', linewidth=0.8, alpha=0.7, label='Sinal Bruto')
axes[0].set_ylabel('Amplitude', fontsize=12)
axes[0].set_title('Sinal Bruto de Vibração', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].legend(loc='upper right')
axes[0].axhline(y=0, color='k', linestyle='--', linewidth=0.5)

# Sinal tratado
axes[1].plot(t_window, signal_treated[:time_window], 'r-', linewidth=0.8, alpha=0.7, label='Sinal Tratado')
axes[1].set_xlabel('Tempo (s)', fontsize=12)
axes[1].set_ylabel('Amplitude', fontsize=12)
axes[1].set_title('Sinal Tratado (Sem Offset, Normalizado e Filtrado)', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend(loc='upper right')
axes[1].axhline(y=0, color='k', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '02_comparacao_bruto_vs_tratado.png'), dpi=DPI, bbox_inches='tight')
print(f"✓ Gráfico salvo: {OUTPUT_DIR}/02_comparacao_bruto_vs_tratado.png")
plt.show()

# Comparação estatística
print("\nComparação Estatística:")
print("-" * 50)
print(f"{'Métrica':<25} {'Bruto':<15} {'Tratado':<15}")
print("-" * 50)
print(f"{'Média':<25} {np.mean(signal_raw):>15.4f} {np.mean(signal_treated):>15.6f}")
print(f"{'Desvio Padrão':<25} {np.std(signal_raw):>15.4f} {np.std(signal_treated):>15.4f}")
print(f"{'Amplitude Máxima':<25} {np.max(np.abs(signal_raw)):>15.4f} {np.max(np.abs(signal_treated)):>15.4f}")
print(f"{'Amplitude Mínima':<25} {np.min(signal_raw):>15.4f} {np.min(signal_treated):>15.4f}")
print("-" * 50)


## 3. Análise Espectral

### 3.1 Transformada de Fourier Rápida (FFT)


In [ ]:
# Aplicar FFT no sinal tratado
N = len(signal_treated)
fft_values = fft(signal_treated)
fft_magnitude = np.abs(fft_values) / N  # Normalização
fft_magnitude_db = 20 * np.log10(fft_magnitude + 1e-10)  # Em decibéis
frequencies = fftfreq(N, 1/SAMPLE_RATE)

# Apenas frequências positivas
positive_freq_idx = frequencies >= 0
freq_positive = frequencies[positive_freq_idx]
magnitude_positive = fft_magnitude[positive_freq_idx]
magnitude_db_positive = fft_magnitude_db[positive_freq_idx]

print(f"✓ FFT calculada com {N} pontos")
print(f"✓ Resolução de frequência: {frequencies[1]:.2f} Hz")
print(f"✓ Frequência máxima analisada: {freq_positive[-1]:.2f} Hz")


### 3.2 Visualização do Espectro de Frequência


In [ ]:
# Visualização do espectro de magnitude
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Espectro linear (até 1000 Hz para melhor visualização)
freq_limit = 1000
freq_mask = freq_positive <= freq_limit

axes[0].plot(freq_positive[freq_mask], magnitude_positive[freq_mask], 'b-', linewidth=1.2)
axes[0].set_xlabel('Frequência (Hz)', fontsize=12)
axes[0].set_ylabel('Magnitude', fontsize=12)
axes[0].set_title('Espectro de Magnitude (Linear)', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim(0, freq_limit)

# Espectro em decibéis
axes[1].plot(freq_positive[freq_mask], magnitude_db_positive[freq_mask], 'r-', linewidth=1.2)
axes[1].set_xlabel('Frequência (Hz)', fontsize=12)
axes[1].set_ylabel('Magnitude (dB)', fontsize=12)
axes[1].set_title('Espectro de Magnitude (dB)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim(0, freq_limit)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '03_espectro_frequencia.png'), dpi=DPI, bbox_inches='tight')
print(f"✓ Gráfico salvo: {OUTPUT_DIR}/03_espectro_frequencia.png")
plt.show()


### 3.3 Identificação de Picos de Frequência


In [ ]:
# Detecção de picos no espectro
# Altura mínima relativa ao pico máximo
height_threshold = np.max(magnitude_positive) * 0.1  # 10% do pico máximo
distance_samples = int(SAMPLE_RATE / 100)  # Mínimo 100 Hz entre picos

peaks, properties = find_peaks(magnitude_positive, 
                               height=height_threshold,
                               distance=distance_samples)

# Ordenar picos por magnitude
peak_frequencies = freq_positive[peaks]
peak_magnitudes = magnitude_positive[peaks]
peak_indices_sorted = np.argsort(peak_magnitudes)[::-1]  # Ordem decrescente

print(f"✓ {len(peaks)} picos identificados no espectro")
print("\nPrincipais frequências identificadas:")
print("-" * 50)
print(f"{'Frequência (Hz)':<20} {'Magnitude':<15} {'Possível Origem':<30}")
print("-" * 50)

# Exibir top 10 picos
for i, idx in enumerate(peak_indices_sorted[:10]):
    freq = peak_frequencies[idx]
    mag = peak_magnitudes[idx]
    # Identificar possível origem (será melhorado na próxima célula)
    origin = "A ser analisado"
    print(f"{freq:>15.2f} Hz {mag:>15.6f} {origin:<30}")
print("-" * 50)


In [ ]:
# Visualização dos picos identificados
fig, ax = plt.subplots(figsize=(14, 6))

freq_limit = 1000
freq_mask = freq_positive <= freq_limit

ax.plot(freq_positive[freq_mask], magnitude_positive[freq_mask], 'b-', linewidth=1.2, alpha=0.7, label='Espectro')
ax.scatter(peak_frequencies[peak_indices_sorted[:10]], 
           peak_magnitudes[peak_indices_sorted[:10]], 
           color='red', s=100, zorder=5, label='Picos Identificados')

# Anotar os top 5 picos
for i, idx in enumerate(peak_indices_sorted[:5]):
    freq = peak_frequencies[idx]
    mag = peak_magnitudes[idx]
    ax.annotate(f'{freq:.1f} Hz', 
                xy=(freq, mag), 
                xytext=(10, 10), 
                textcoords='offset points',
                fontsize=9,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

ax.set_xlabel('Frequência (Hz)', fontsize=12)
ax.set_ylabel('Magnitude', fontsize=12)
ax.set_title('Espectro de Frequência com Picos Identificados', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_xlim(0, freq_limit)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '04_espectro_com_picos.png'), dpi=DPI, bbox_inches='tight')
print(f"✓ Gráfico salvo: {OUTPUT_DIR}/04_espectro_com_picos.png")
plt.show()


### 3.4 Discussão Física das Frequências Identificadas

As frequências de vibração em rolamentos podem ser relacionadas a diferentes fenômenos físicos:


In [ ]:
# Função para identificar possíveis origens das frequências
def identificar_origem_frequencia(freq, freq_rotacao=30):
    """
    Identifica possível origem de uma frequência baseado em valores típicos
    de falhas em rolamentos.
    
    Parâmetros típicos para rolamentos:
    - BPFO (Ball Pass Frequency Outer): ~3.58 × freq_rotacao
    - BPFI (Ball Pass Frequency Inner): ~5.41 × freq_rotacao
    - BSF (Ball Spin Frequency): ~2.36 × freq_rotacao
    - FTF (Fundamental Train Frequency): ~0.4 × freq_rotacao
    """
    tolerancia = 5  # Hz de tolerância
    
    # Frequência de rotação e harmônicos
    if abs(freq - freq_rotacao) < tolerancia:
        return "Rotação (1x)"
    for h in [2, 3, 4]:
        if abs(freq - h * freq_rotacao) < tolerancia:
            return f"Rotacao ({h}x harmônico)"
    
    # Frequências de falhas em rolamentos (valores típicos)
    bpfo = 3.58 * freq_rotacao
    bpfi = 5.41 * freq_rotacao
    bsf = 2.36 * freq_rotacao
    ftf = 0.4 * freq_rotacao
    
    if abs(freq - bpfo) < tolerancia:
        return "BPFO (Falha pista externa)"
    if abs(freq - bpfi) < tolerancia:
        return "BPFI (Falha pista interna)"
    if abs(freq - bsf) < tolerancia:
        return "BSF (Falha esfera)"
    if abs(freq - ftf) < tolerancia:
        return "FTF (Frequência gaiola)"
    
    # Ruído de linha (60 Hz)
    if abs(freq - 60) < 2:
        return "Ruído de linha (60 Hz)"
    
    # Harmônicos de componentes conhecidas
    for base in [bpfo, bpfi, bsf]:
        for h in [2, 3]:
            if abs(freq - h * base) < tolerancia:
                return f"Harmônico de falha ({h}x)"
    
    return "Frequência não identificada"

# Análise detalhada dos picos
print("Análise Detalhada das Frequências Identificadas:")
print("=" * 80)
print(f"{'Frequência (Hz)':<20} {'Magnitude':<15} {'Origem Provável':<40}")
print("=" * 80)

freq_rotacao_estimada = 30  # Ajuste conforme necessário

for i, idx in enumerate(peak_indices_sorted[:15]):
    freq = peak_frequencies[idx]
    mag = peak_magnitudes[idx]
    origem = identificar_origem_frequencia(freq, freq_rotacao_estimada)
    print(f"{freq:>15.2f} Hz {mag:>15.6f} {origem:<40}")

print("=" * 80)
print("\nLegenda:")
print("  BPFO: Ball Pass Frequency Outer (Falha na pista externa)")
print("  BPFI: Ball Pass Frequency Inner (Falha na pista interna)")
print("  BSF:  Ball Spin Frequency (Falha na esfera)")
print("  FTF:  Fundamental Train Frequency (Frequência da gaiola)")
print("\nNOTA: Esta análise é baseada em valores típicos. Ajuste os parâmetros")
print("      conforme as especificações do seu rolamento específico.")


In [ ]:
# Cálculo das features principais
def calcular_rms(sinal):
    """Calcula o Root Mean Square (RMS) do sinal"""
    return np.sqrt(np.mean(sinal**2))

def calcular_valor_pico(sinal):
    """Calcula o valor de pico (amplitude máxima)"""
    return np.max(np.abs(sinal))

def calcular_fator_crista(sinal):
    """Calcula o Fator de Crista (Crest Factor) = Peak / RMS"""
    peak = calcular_valor_pico(sinal)
    rms = calcular_rms(sinal)
    return peak / rms if rms > 0 else 0

def calcular_kurtosis(sinal):
    """Calcula a Curtose (Kurtosis) - medida de "picosidade" do sinal"""
    from scipy.stats import kurtosis
    return kurtosis(sinal, fisher=False)  # Fisher=False para excesso + 3

def calcular_skewness(sinal):
    """Calcula a Assimetria (Skewness)"""
    from scipy.stats import skew
    return skew(sinal)

def calcular_fator_forma(sinal):
    """Calcula o Fator de Forma = RMS / Valor Médio Absoluto"""
    rms = calcular_rms(sinal)
    mean_abs = np.mean(np.abs(sinal))
    return rms / mean_abs if mean_abs > 0 else 0

# Calcular features para o sinal tratado
features = {
    'RMS': calcular_rms(signal_treated),
    'Valor de Pico': calcular_valor_pico(signal_treated),
    'Fator de Crista': calcular_fator_crista(signal_treated),
    'Kurtosis': calcular_kurtosis(signal_treated),
    'Skewness': calcular_skewness(signal_treated),
    'Fator de Forma': calcular_fator_forma(signal_treated)
}

print("Features Extraídas do Sinal:")
print("=" * 60)
for nome, valor in features.items():
    print(f"{nome:<25} {valor:>15.6f}")
print("=" * 60)


In [ ]:
# Visualização das features em tabela e gráfico
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Tabela de features
feature_names = list(features.keys())
feature_values = list(features.values())

# Gráfico de barras
ax1.barh(feature_names, feature_values, color='steelblue', alpha=0.7)
ax1.set_xlabel('Valor', fontsize=11)
ax1.set_title('Features Extraídas', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='x')

# Tabela numérica
ax2.axis('tight')
ax2.axis('off')
table_data = [[f"{v:.6f}"] for v in feature_values]
table = ax2.table(cellText=table_data,
                  rowLabels=feature_names,
                  colLabels=['Valor'],
                  cellLoc='center',
                  loc='center',
                  bbox=[0, 0, 1, 1])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)
ax2.set_title('Valores das Features', fontsize=13, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '05_features_extraidas.png'), dpi=DPI, bbox_inches='tight')
print(f"✓ Gráfico salvo: {OUTPUT_DIR}/05_features_extraidas.png")
plt.show()


### Interpretação das Features

#### RMS (Root Mean Square)
- **Significado**: Medida da energia efetiva do sinal
- **Aplicação**: Indicador geral do nível de vibração
- **Uso em IoT**: Monitoramento contínuo de condição; aumento progressivo indica degradação

#### Valor de Pico
- **Significado**: Amplitude máxima instantânea
- **Aplicação**: Detecta impactos e eventos transientes
- **Uso em IoT**: Alerta imediato para eventos anômalos

#### Fator de Crista (Crest Factor)
- **Significado**: Razão entre pico e RMS (normalmente 1.4 para sinal senoidal puro)
- **Aplicação**: 
  - Valores baixos (< 2): Sinal suave, operação normal
  - Valores altos (> 3): Presença de picos agudos, possível falha incipiente
- **Uso em IoT**: Indicador precoce de falhas em rolamentos

#### Kurtosis
- **Significado**: Mede a "picosidade" da distribuição
- **Aplicação**: 
  - Valores próximos a 3: Distribuição normal
  - Valores > 3: Sinal com muitos picos (possível falha)
- **Uso em IoT**: Detecção de falhas em estágio inicial


In [ ]:
# Análise comparativa: Features do sinal bruto vs tratado
features_bruto = {
    'RMS': calcular_rms(signal_raw),
    'Valor de Pico': calcular_valor_pico(signal_raw),
    'Fator de Crista': calcular_fator_crista(signal_raw),
}

features_tratado = {
    'RMS': calcular_rms(signal_treated),
    'Valor de Pico': calcular_valor_pico(signal_treated),
    'Fator de Crista': calcular_fator_crista(signal_treated),
}

# Tabela comparativa
print("\nComparação de Features: Sinal Bruto vs Tratado")
print("=" * 70)
print(f"{'Feature':<20} {'Bruto':<15} {'Tratado':<15} {'Diferença (%)':<15}")
print("=" * 70)

for feature in features_bruto.keys():
    valor_bruto = features_bruto[feature]
    valor_tratado = features_tratado[feature]
    diferenca_pct = ((valor_tratado - valor_bruto) / valor_bruto * 100) if valor_bruto != 0 else 0
    print(f"{feature:<20} {valor_bruto:>15.6f} {valor_tratado:>15.6f} {diferenca_pct:>15.2f}%")
print("=" * 70)


## 5. Aplicações em Monitoramento Contínuo via IoT

As features extraídas podem ser utilizadas em sistemas de monitoramento contínuo:

### Vantagens para IoT:
1. **Baixo custo computacional**: Features são valores escalares (não precisam transmitir todo o sinal)
2. **Detecção em tempo real**: Cálculo rápido permite análise contínua
3. **Integração com IA**: Features podem alimentar modelos de ML para classificação/predição
4. **Redução de banda**: Transmissão apenas de features, não do sinal completo
5. **Edge Computing**: Processamento local antes do envio para nuvem


In [ ]:
# Visualização resumo: Sinal, Espectro e Features
fig = plt.figure(figsize=(16, 10))

# Layout de subplots
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# 1. Sinal no tempo (tratado)
ax1 = fig.add_subplot(gs[0, :])
time_window = int(SAMPLE_RATE * 0.2)
ax1.plot(t[:time_window], signal_treated[:time_window], 'b-', linewidth=0.8)
ax1.set_xlabel('Tempo (s)', fontsize=11)
ax1.set_ylabel('Amplitude', fontsize=11)
ax1.set_title('Sinal Tratado no Tempo', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)

# 2. Espectro de frequência
ax2 = fig.add_subplot(gs[1, :])
freq_limit = 500
freq_mask = freq_positive <= freq_limit
ax2.plot(freq_positive[freq_mask], magnitude_positive[freq_mask], 'r-', linewidth=1.2)
ax2.scatter(peak_frequencies[peak_indices_sorted[:5]], 
           peak_magnitudes[peak_indices_sorted[:5]], 
           color='red', s=80, zorder=5)
ax2.set_xlabel('Frequência (Hz)', fontsize=11)
ax2.set_ylabel('Magnitude', fontsize=11)
ax2.set_title('Espectro de Frequência', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

# 3. Features principais
ax3 = fig.add_subplot(gs[2, 0])
main_features = ['RMS', 'Valor de Pico', 'Fator de Crista']
main_values = [features[f] for f in main_features]
ax3.barh(main_features, main_values, color=['steelblue', 'orange', 'green'], alpha=0.7)
ax3.set_xlabel('Valor', fontsize=11)
ax3.set_title('Features Principais', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3, axis='x')

# 4. Features adicionais
ax4 = fig.add_subplot(gs[2, 1])
extra_features = ['Kurtosis', 'Skewness', 'Fator de Forma']
extra_values = [features[f] for f in extra_features]
ax4.barh(extra_features, extra_values, color=['purple', 'brown', 'teal'], alpha=0.7)
ax4.set_xlabel('Valor', fontsize=11)
ax4.set_title('Features Adicionais', fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3, axis='x')

plt.suptitle('Resumo Completo da Análise de Vibração', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '06_resumo_completo.png'), dpi=DPI, bbox_inches='tight')
print(f"✓ Gráfico salvo: {OUTPUT_DIR}/06_resumo_completo.png")
plt.show()


## 6. Conclusões

### Resultados Principais:
1. **Tratamento do Sinal**: Remoção de offset, normalização e filtragem foram aplicadas com sucesso
2. **Análise Espectral**: Identificadas frequências características que podem indicar condições específicas do rolamento
3. **Features Extraídas**: Indicadores quantitativos calculados para monitoramento contínuo

### Próximos Passos (para aplicação real):
- Coleta de dados históricos para estabelecer baseline
- Definição de thresholds para cada feature
- Implementação de sistema de alertas baseado em features
- Integração com modelos de Machine Learning para classificação de condições
- Deploy em plataforma IoT com edge computing

### Vínculo com Indústria 4.0:
Este tipo de análise é fundamental para:
- **Sensores Inteligentes**: Processamento local de sinais
- **Manutenção Preditiva**: Detecção precoce de falhas
- **Digital Twin**: Modelagem virtual do equipamento
- **DataOps Industrial**: Tomada de decisão baseada em dados

---
**Nota**: Este notebook demonstra o fluxo completo de análise. Em aplicações reais, 
ajuste os parâmetros conforme as especificações do seu equipamento e dataset.
